In [10]:

%reset

import xarray as xr
import glob
import numpy as np
import netCDF4 as nc
import warnings
import matplotlib.path as mpath
import cartopy.feature as cfeature

warnings.filterwarnings('ignore')
from matplotlib import pyplot as plt
from cmocean import cm
import warnings
warnings.filterwarnings('ignore')
import cartopy.crs as ccrs
import sys
sys.path.append('/gpfs/home/mep22dku/scratch/SOZONE/MO_pipeline/')
sys.path.append('/gpfs/home/mep22dku/scratch/SOZONE/UTILS/')
import scendict as sc
import utils as ut
import time


Once deleted, variables cannot be recovered. Proceed (y/[n])?  y


In [11]:
tmesh = xr.open_dataset('/gpfs/data/greenocean/software/resources/regrid/mesh_mask3_6.nc')
tmesh['csize'] = tmesh.tmask[0,0,:,:] * tmesh.e1t[0,:,:] * tmesh.e2t[0,:,:]

so_csize = (tmesh['csize'][0:37,:].values)
so_csize_broad = np.zeros([365*10, 37, 182])

for i in range(0,365*10):
    so_csize_broad[i,:,:] = so_csize

## mapped extraction

In [12]:
def make_yearlist(yrst, yrend, dtype, tr, baseDir):
    yrs = np.arange(yrst,yrend+1,1)
    ylist = []
    for i in range(0,len(yrs)):
        ty = f'{baseDir}/{tr}/ORCA2_1m_{yrs[i]}*{dtype}*.nc'
        t2 = glob.glob(ty)
        #print(t2)
        ylist.append(t2[0])
    return ylist

def get_DICTA_maps(rundir, run, ystart = 1998, yend = 2010, clq = False):
    
    tmesh = xr.open_dataset('/gpfs/data/greenocean/software/resources/regrid/mesh_mask3_6.nc')
    tmesh['csize'] = tmesh.tmask[0,0,:,:] * tmesh.e1t[0,:,:] * tmesh.e2t[0,:,:]
    so_csize = (tmesh['csize'][0:37,:].values)
    ################################
    if clq == False:
        noyrs = yend - ystart + 1
        ylist = make_yearlist(ystart,yend,'ptrc_T', run, rundir)
        w = xr.open_mfdataset(ylist)
        yrly_DIC_surf = np.zeros([noyrs,12,37,182])
        yrly_TA_surf = np.zeros([noyrs,12,37,182])
        for i in range(0,noyrs):
            ind = i*12
            yrly_DIC_surf[i,:,:,:] = w['DIC'][ind:ind+12,0,0:37,:]
            yrly_TA_surf[i,:,:,:] = w['Alkalini'][ind:ind+12,0,0:37,:]

        yrly_DIC_surf = np.nanmean(yrly_DIC_surf, axis = 0)
        yrly_TA_surf = np.nanmean(yrly_TA_surf, axis = 0)
    
    if clq == True:
        tnc = f'/gpfs/home/e415/scratch/TOM10_CL_{run}/{run}_ptrc_1998_2010_ave.nc'
        w = xr.open_dataset(tnc)
        yrly_DIC_surf = w['DIC'][:,0,0:37,:].values
        yrly_TA_surf = w['Alkalini'][:,0,0:37,:].values
   
    ## nans 

   ####################################     
    summer_DIC_surf = (yrly_DIC_surf[0,:,:]*31+yrly_DIC_surf[1,:,:]*28+yrly_DIC_surf[11,:,:]*31)/90
    winter_DIC_surf = (yrly_DIC_surf[5,:,:]*30+yrly_DIC_surf[6,:,:]*31+yrly_DIC_surf[7,:,:]*31)/92
    summer_TA_surf = (yrly_TA_surf[0,:,:]*31+yrly_TA_surf[1,:,:]*28+yrly_TA_surf[11,:,:]*31)/90
    winter_TA_surf = (yrly_TA_surf[5,:,:]*30+yrly_TA_surf[6,:,:]*31+yrly_TA_surf[7,:,:]*31)/92
    
    DIC_yr = np.zeros([12])
    TA_yr = np.zeros([12])
    for i in range(0,12):
        DIC_yr[i] = np.ma.average(yrly_DIC_surf[i,:,:], weights = so_csize)
        TA_yr[i] = np.ma.average(yrly_TA_surf[i,:,:], weights = so_csize)
    
    ds2 = xr.Dataset(
    {
        "DIC": (["time"], DIC_yr*1e6, {"units": "umol/L"}),
        "TA": (["time"], TA_yr*1e6, {"units": "umol/L"}),
        "DIC_summer": (["y","x"], summer_DIC_surf*1e6, {"units": "umol/L"}),
        "DIC_winter": (["y","x"], winter_DIC_surf*1e6, {"units": "umol/L"}),
        "TA_summer": (["y","x"], summer_TA_surf*1e6, {"units": "umol/L"}),
        "TA_winter": (["y","x"], winter_TA_surf*1e6, {"units": "umol/L"}),
    },
    coords={
        "month": (["time"], np.arange(1,13,1)),
        "y": (["y"], np.arange(0,37,1)),
        "x": (["x"], np.arange(0,182,1))
    },
    attrs=dict(description="subset at index 0 is whole SO\
        (mod.ind. 0-37, latitude south of -50),\
        at index 1 is -65 - -55 (mod.ind 22:33)"),
    )
    
    tnam = f'./ncs/SOmap_DICTA_{run}_{ystart}_{yend}.nc'
    if clq:
        tnam = f'./ncs/SOmap_DICTA_TOM10_CL_{run}_{ystart}_{yend}.nc'
    #ds2.to_netcdf(f'test3.nc')
    print('made u a netcdf bb:')
    print(tnam)
    ds2.to_netcdf(tnam)
    return 

word = 'test1'
def wrapper(rundir, run, ystart = 1998, yend = 2010, clq = False):

    time1 = time.time()
    get_DICTA_maps(rundir, run, ystart, yend, clq)
    time2 = time.time()
    print(time2-time1)
    
rundir = '/gpfs/data/greenocean/software/runs/'
run = 'TOM12_TJ_1ASA'
# wrapper(rundir, run, 1998, 2010)

In [13]:
horse = False
if horse:
    rundir = '/gpfs/data/greenocean/software/runs/'
    run = 'TOM12_TJ_1ASA'
    wrapper(rundir, run, 1998, 2010)
    run = 'TOM12_DW_WE43'
    wrapper(rundir, run, 1998, 2010)
    run = 'TOM12_DW_GA01'
    wrapper(rundir, run, 1998, 2010)
# horse = True
if horse:
    run = 'LIG9'
    wrapper(rundir, run, 1998, 2010, clq = True)
    run = 'LIGD'    
    wrapper(rundir, run, 1998, 2010, clq = True)
    
    rundir = '/gpfs/home/hnb15vcu/scratch/ModelRuns/'
    run = 'TOM12_RW_3G20'
    wrapper(rundir, run, 1955, 1960)
    run = 'TOM12_RW_3G07'
    wrapper(rundir, run, 1955, 1960)
    run = 'TOM12_RW_3G09'
    wrapper(rundir, run, 1955, 1960)
    run = 'TOM12_RW_3G35'
    wrapper(rundir, run, 1955, 1960)

In [16]:
def get_saltemp_maps(rundir, run, ystart = 1998, yend = 2010, clq = False):
    
    tmesh = xr.open_dataset('/gpfs/data/greenocean/software/resources/regrid/mesh_mask3_6.nc')
    tmesh['csize'] = tmesh.tmask[0,0,:,:] * tmesh.e1t[0,:,:] * tmesh.e2t[0,:,:]
    so_csize = (tmesh['csize'][0:37,:].values)
    ################################
    if clq == False:
        noyrs = yend - ystart + 1
        ylist = make_yearlist(ystart,yend,'grid_T', run, rundir)
        w = xr.open_mfdataset(ylist)
        yrly_sal_surf = np.zeros([noyrs,12,37,182])
        yrly_temp_surf = np.zeros([noyrs,12,37,182])
        for i in range(0,noyrs):
            ind = i*12
            yrly_sal_surf[i,:,:,:] = w['vosaline'][ind:ind+12,0,0:37,:]
            yrly_temp_surf[i,:,:,:] = w['votemper'][ind:ind+12,0,0:37,:]

        yrly_sal_surf = np.nanmean(yrly_sal_surf, axis = 0)
        yrly_temp_surf = np.nanmean(yrly_temp_surf, axis = 0)
    
    if clq == True:
        tnc = f'/gpfs/home/e415/scratch/TOM10_CL_{run}/{run}_ptrc_1998_2010_ave.nc'
        w = xr.open_dataset(tnc)
        yrly_sal_surf = w['sal'][:,0,0:37,:].values
        yrly_temp_surf = w['Alkalini'][:,0,0:37,:].values
   
    ## nans 

   ####################################     
    summer_sal_surf = (yrly_sal_surf[0,:,:]*31+yrly_sal_surf[1,:,:]*28+yrly_sal_surf[11,:,:]*31)/90
    winter_sal_surf = (yrly_sal_surf[5,:,:]*30+yrly_sal_surf[6,:,:]*31+yrly_sal_surf[7,:,:]*31)/92
    summer_temp_surf = (yrly_temp_surf[0,:,:]*31+yrly_temp_surf[1,:,:]*28+yrly_temp_surf[11,:,:]*31)/90
    winter_temp_surf = (yrly_temp_surf[5,:,:]*30+yrly_temp_surf[6,:,:]*31+yrly_temp_surf[7,:,:]*31)/92
    
    sal_yr = np.zeros([12])
    temp_yr = np.zeros([12])
    for i in range(0,12):
        sal_yr[i] = np.ma.average(yrly_sal_surf[i,:,:], weights = so_csize)
        temp_yr[i] = np.ma.average(yrly_temp_surf[i,:,:], weights = so_csize)
    
    ds2 = xr.Dataset(
    {
        "sal": (["time"], sal_yr, {"units": "(standard)"}),
        "temp": (["time"], temp_yr, {"units": "(standard)"}),
        "sal_summer": (["y","x"], summer_sal_surf, {"units": "(standard)"}),
        "sal_winter": (["y","x"], winter_sal_surf, {"units": "(standard)"}),
        "temp_summer": (["y","x"], summer_temp_surf, {"units": "(standard)"}),
        "temp_winter": (["y","x"], winter_temp_surf, {"units": "(standard)"}),
    },
    coords={
        "month": (["time"], np.arange(1,13,1)),
        "y": (["y"], np.arange(0,37,1)),
        "x": (["x"], np.arange(0,182,1))
    },
    attrs=dict(description=""),
    )
    
    tnam = f'./ncs/SOmap_saltemp_{run}_{ystart}_{yend}.nc'
    if clq:
        tnam = f'./ncs/SOmap_saltemp_TOM10_CL_{run}_{ystart}_{yend}.nc'
    #ds2.to_netcdf(f'test3.nc')
    print('made u a netcdf bb:')
    print(tnam)
    ds2.to_netcdf(tnam)
    return 

word = 'test1'
def wrapper_st(rundir, run, ystart = 1998, yend = 2010, clq = False):

    time1 = time.time()
    get_saltemp_maps(rundir, run, ystart, yend, clq)
    time2 = time.time()
    print(time2-time1)
    

In [17]:
horse = True
if horse:
    rundir = '/gpfs/data/greenocean/software/runs/'
    run = 'TOM12_TJ_1ASA'
    wrapper_st(rundir, run, 1998, 2010)
    run = 'TOM12_DW_WE43'
    wrapper_st(rundir, run, 1998, 2010)
    run = 'TOM12_DW_GA01'
    wrapper_st(rundir, run, 1998, 2010)
horse = True
if horse:
    # run = 'LIG9'
    # wrapper_st(rundir, run, 1998, 2010, clq = True)
    # run = 'LIGD'    
    # wrapper_st(rundir, run, 1998, 2010, clq = True)
    
    rundir = '/gpfs/home/hnb15vcu/scratch/ModelRuns/'
    run = 'TOM12_RW_3G20'
    wrapper_st(rundir, run, 1955, 1960)
    run = 'TOM12_RW_3G07'
    wrapper_st(rundir, run, 1955, 1960)
    run = 'TOM12_RW_3G09'
    wrapper_st(rundir, run, 1955, 1960)
    run = 'TOM12_RW_3G35'
    wrapper_st(rundir, run, 1955, 1960)

made u a netcdf bb:
./ncs/SOmap_saltemp_TOM12_TJ_1ASA_1998_2010.nc
18.660070180892944
made u a netcdf bb:
./ncs/SOmap_saltemp_TOM12_DW_WE43_1998_2010.nc
24.91701912879944
made u a netcdf bb:
./ncs/SOmap_saltemp_TOM12_DW_GA01_1998_2010.nc
25.90563154220581
made u a netcdf bb:
./ncs/SOmap_saltemp_TOM12_RW_3G20_1955_1960.nc
10.202632904052734
made u a netcdf bb:
./ncs/SOmap_saltemp_TOM12_RW_3G07_1955_1960.nc
9.509237051010132
made u a netcdf bb:
./ncs/SOmap_saltemp_TOM12_RW_3G09_1955_1960.nc
10.335500240325928
made u a netcdf bb:
./ncs/SOmap_saltemp_TOM12_RW_3G35_1955_1960.nc
11.277254581451416


In [23]:
def get_pco2_maps(rundir, run, ystart = 1998, yend = 2010, clq = False):
    
    tmesh = xr.open_dataset('/gpfs/data/greenocean/software/resources/regrid/mesh_mask3_6.nc')
    tmesh['csize'] = tmesh.tmask[0,0,:,:] * tmesh.e1t[0,:,:] * tmesh.e2t[0,:,:]
    so_csize = (tmesh['csize'][0:37,:].values)
    ################################
    if clq == False:
        noyrs = yend - ystart + 1
        ylist = make_yearlist(ystart,yend,'diad_T', run, rundir)
        w = xr.open_mfdataset(ylist)
        yrly_pco2_surf = np.zeros([noyrs,12,37,182])
        yrly_dpco2_surf = np.zeros([noyrs,12,37,182])
        for i in range(0,noyrs):
            ind = i*12
            yrly_pco2_surf[i,:,:,:] = w['pCO2'][ind:ind+12,0:37,:]
            yrly_dpco2_surf[i,:,:,:] = w['dpCO2'][ind:ind+12,0:37,:]

        yrly_pco2_surf = np.nanmean(yrly_pco2_surf, axis = 0)
        yrly_dpco2_surf = np.nanmean(yrly_dpco2_surf, axis = 0)
    
    if clq == True:
        tnc = f'/gpfs/home/e415/scratch/TOM10_CL_{run}/{run}_dia2d_1998_2010_ave.nc'
        w = xr.open_dataset(tnc)
        yrly_pco2_surf = w['dpco2'][:,0:37,:].values
        yrly_dpco2_surf =  w['dpco2'][:,0:37,:].values
        yrly_dpco2_surf[:] = np.nan
    ## nans 

   ####################################     
    summer_pco2_surf = (yrly_pco2_surf[0,:,:]*31+yrly_pco2_surf[1,:,:]*28+yrly_pco2_surf[11,:,:]*31)/90
    winter_pco2_surf = (yrly_pco2_surf[5,:,:]*30+yrly_pco2_surf[6,:,:]*31+yrly_pco2_surf[7,:,:]*31)/92
    summer_dpco2_surf = (yrly_dpco2_surf[0,:,:]*31+yrly_dpco2_surf[1,:,:]*28+yrly_dpco2_surf[11,:,:]*31)/90
    winter_dpco2_surf = (yrly_dpco2_surf[5,:,:]*30+yrly_dpco2_surf[6,:,:]*31+yrly_dpco2_surf[7,:,:]*31)/92
    
    pco2_yr = np.zeros([12])
    dpco2_yr = np.zeros([12])
    for i in range(0,12):
        pco2_yr[i] = np.ma.average(yrly_pco2_surf[i,:,:], weights = so_csize)
        dpco2_yr[i] = np.ma.average(yrly_dpco2_surf[i,:,:], weights = so_csize)
    
    ds2 = xr.Dataset(
    {
        "pco2": (["time"], pco2_yr, {"units": "(standard)"}),
        "dpco2": (["time"], dpco2_yr, {"units": "(standard)"}),
        "pco2_summer": (["y","x"], summer_pco2_surf, {"units": "(standard)"}),
        "pco2_winter": (["y","x"], winter_pco2_surf, {"units": "(standard)"}),
        "dpco2_summer": (["y","x"], summer_dpco2_surf, {"units": "(standard)"}),
        "dpco2_winter": (["y","x"], winter_dpco2_surf, {"units": "(standard)"}),
    },
    coords={
        "month": (["time"], np.arange(1,13,1)),
        "y": (["y"], np.arange(0,37,1)),
        "x": (["x"], np.arange(0,182,1))
    },
    attrs=dict(description=""),
    )
    
    tnam = f'./ncs/SOmap_pco2_{run}_{ystart}_{yend}.nc'
    if clq:
        tnam = f'./ncs/SOmap_pco2_TOM10_CL_{run}_{ystart}_{yend}.nc'
    #ds2.to_netcdf(f'test3.nc')
    print('made u a netcdf bb:')
    print(tnam)
    ds2.to_netcdf(tnam)
    return 

def wrapper_pc(rundir, run, ystart = 1998, yend = 2010, clq = False):

    time1 = time.time()
    get_pco2_maps(rundir, run, ystart, yend, clq)
    time2 = time.time()
    print(time2-time1)
    

In [25]:
horse = False
if horse:
    rundir = '/gpfs/data/greenocean/software/runs/'
    run = 'TOM12_TJ_1ASA'
    wrapper_pc(rundir, run, 1998, 2010)
    run = 'TOM12_DW_WE43'
    wrapper_pc(rundir, run, 1998, 2010)
    run = 'TOM12_DW_GA01'
    wrapper_pc(rundir, run, 1998, 2010)
horse = True
if horse:
    run = 'LIG9'
    wrapper_pc(rundir, run, 1998, 2010, clq = True)
    run = 'LIGD'    
    wrapper_pc(rundir, run, 1998, 2010, clq = True)
    
    rundir = '/gpfs/home/hnb15vcu/scratch/ModelRuns/'
    run = 'TOM12_RW_3G20'
    wrapper_pc(rundir, run, 1955, 1960)
    run = 'TOM12_RW_3G07'
    wrapper_pc(rundir, run, 1955, 1960)
    run = 'TOM12_RW_3G09'
    wrapper_pc(rundir, run, 1955, 1960)
    run = 'TOM12_RW_3G35'
    wrapper_pc(rundir, run, 1955, 1960)

made u a netcdf bb:
./ncs/SOmap_pco2_TOM10_CL_LIG9_1998_2010.nc
0.7627184391021729
made u a netcdf bb:
./ncs/SOmap_pco2_TOM10_CL_LIGD_1998_2010.nc
1.226266622543335
made u a netcdf bb:
./ncs/SOmap_pco2_TOM12_RW_3G20_1955_1960.nc
6.692358493804932
made u a netcdf bb:
./ncs/SOmap_pco2_TOM12_RW_3G07_1955_1960.nc
6.543676853179932
made u a netcdf bb:
./ncs/SOmap_pco2_TOM12_RW_3G09_1955_1960.nc
8.65129804611206
made u a netcdf bb:
./ncs/SOmap_pco2_TOM12_RW_3G35_1955_1960.nc
7.476395845413208


In [45]:
def calc_200m_ppt(ppt):
    '''
    take one year of PPT values - size 12x31x149x182
    '''
    ### appropriate masking
    tmesh = xr.open_dataset('/gpfs/data/greenocean/software/resources/regrid/mesh_mask3_6.nc')
    tmesh['csize'] = tmesh.tmask[0,0,:,:] * tmesh.e1t[0,:,:] * tmesh.e2t[0,:,:]
    deptat = (tmesh['gdept_1d'][0,:])
    tmask = (tmesh['tmask'])

    depth4d = np.zeros([12,31,149,182])
    tmask4d = np.zeros([12,31,149,182])
    #broadcast m depth across the whole array to get an array of "depth of cell here in the grid" that is the same shape as a given variable
    for i in range(0,31):
        depth4d[:,i,:,:] = deptat[i].values

    for i in range(0,12):
        tmask4d[i,:,:,:] = tmask
    depth4dm = depth4d *tmask4d 
    #######
    
    PPT_molC_m2_s = ppt *depth4dm
    sid = 60*60*24
    PPT_molC_m2_d_top200m = np.nansum(PPT_molC_m2_s[:,0:16,0:37,:], axis = 1)*sid
    PPT_molC_m2_d_top200m[PPT_molC_m2_d_top200m == 0] = np.nan
    # get explosions out
    PPT_molC_m2_d_top200m[PPT_molC_m2_d_top200m > 1e10] = np.nan
    # print(np.shape(PPT_molC_m2_d_top200m))
#     #####
#     PPT_molC_m2_d_top200m_seas_SO = np.zeros([12])
#     for i in range(0,12):
#         simple_prod = PPT_molC_m2_d_top200m[i,0:37,:]
#         ma = np.ma.MaskedArray(simple_prod, mask=np.isnan(simple_prod))
#         PPT_molC_m2_d_top200m_seas_SO[i] = np.ma.average(ma, weights=tmesh['csize'][0:37,:])
    
    return PPT_molC_m2_d_top200m

In [72]:




def get_ppt_maps(rundir, run, ystart = 1998, yend = 2010, clq = False):
    
    tmesh = xr.open_dataset('/gpfs/data/greenocean/software/resources/regrid/mesh_mask3_6.nc')
    tmesh['csize'] = tmesh.tmask[0,0,:,:] * tmesh.e1t[0,:,:] * tmesh.e2t[0,:,:]
    so_csize = (tmesh['csize'][0:37,:].values)
    sid = 60*60*24
    ################################
    if clq == False:
        noyrs = yend - ystart + 1
        ylist = make_yearlist(ystart,yend,'diad_T', run, rundir)
        w = xr.open_mfdataset(ylist)
        yrly_ppt_surf = np.zeros([noyrs,12,37,182])
        yrly_EXP_surf = np.zeros([noyrs,12,37,182])
        for i in range(0,noyrs):
            ind = i*12
            t_ppt = w['PPT'][ind:ind+12,:,:,:]
            PPT_molC_m2_d_top200m = calc_200m_ppt(t_ppt)
            yrly_ppt_surf[i,:,:,:] = PPT_molC_m2_d_top200m
            yrly_EXP_surf[i,:,:,:] = w['EXP'][ind:ind+12,10,0:37,:]

        yrly_ppt_surf = np.nanmean(yrly_ppt_surf, axis = 0)
        yrly_EXP_surf = np.nanmean(yrly_EXP_surf, axis = 0) *sid
    
    if clq == True:
        tnc = f'/gpfs/home/e415/scratch/TOM10_CL_{run}/{run}_dia3d_1998_2010_ave.nc'
        w = xr.open_dataset(tnc)
        t_ppt = w['PPT'][:,:,:,:]
        PPT_molC_m2_d_top200m = calc_200m_ppt(t_ppt)
        yrly_ppt_surf = PPT_molC_m2_d_top200m
        yrly_EXP_surf =  w['EXP'][:,10,0:37,:].values *sid

    ## nans 

   ####################################     
    summer_ppt_surf = (yrly_ppt_surf[0,:,:]*31+yrly_ppt_surf[1,:,:]*28+yrly_ppt_surf[11,:,:]*31)/90
    winter_ppt_surf = (yrly_ppt_surf[5,:,:]*30+yrly_ppt_surf[6,:,:]*31+yrly_ppt_surf[7,:,:]*31)/92
    summer_EXP_surf = (yrly_EXP_surf[0,:,:]*31+yrly_EXP_surf[1,:,:]*28+yrly_EXP_surf[11,:,:]*31)/90
    winter_EXP_surf = (yrly_EXP_surf[5,:,:]*30+yrly_EXP_surf[6,:,:]*31+yrly_EXP_surf[7,:,:]*31)/92
    
    ppt_yr = np.zeros([12])
    EXP_yr = np.zeros([12])
    for i in range(0,12):
        #get mean
        ma = np.ma.MaskedArray(yrly_ppt_surf[i,:,:], mask=np.isnan(yrly_ppt_surf[i,:,:]))
        q = np.ma.average(ma, weights=so_csize)
        ppt_yr[i] = q
        bar = 1e-5*sid
        yrly_EXP_surf[yrly_EXP_surf>bar] = np.nan
        ma = np.ma.MaskedArray(yrly_EXP_surf[i,:,:], mask=np.isnan(yrly_EXP_surf[i,:,:]))
        q = np.ma.average(ma, weights=so_csize)
        EXP_yr[i] = q
    
    print(ppt_yr)
    print(EXP_yr)
    ds2 = xr.Dataset(
    {
        "ppt": (["time"], ppt_yr, {"units": "(mol/m2/day top 200)"}),
        "EXP": (["time"], EXP_yr, {"units": "(mol/m2/day)"}),
        "ppt_summer": (["y","x"], summer_ppt_surf, {"units": "(mol/m2/day) top 200 "}),
        "ppt_winter": (["y","x"], winter_ppt_surf, {"units": "(mol/m2/day) top 200"}),
        "EXP_summer": (["y","x"], summer_EXP_surf, {"units": "(mol/m2/day)"}),
        "EXP_winter": (["y","x"], winter_EXP_surf, {"units": "(mol/m2/day)"}),
    },
    coords={
        "month": (["time"], np.arange(1,13,1)),
        "y": (["y"], np.arange(0,37,1)),
        "x": (["x"], np.arange(0,182,1))
    },
    attrs=dict(description=""),
    )
    
    tnam = f'./ncs/SOmap_ppt_{run}_{ystart}_{yend}.nc'
    if clq:
        tnam = f'./ncs/SOmap_ppt_TOM10_CL_{run}_{ystart}_{yend}.nc'
    #ds2.to_netcdf(f'test3.nc')
    print('made u a netcdf bb:')
    print(tnam)
    ds2.to_netcdf(tnam)
    return 

def wrapper_ppt(rundir, run, ystart = 1998, yend = 2010, clq = False):

    time1 = time.time()
    get_ppt_maps(rundir, run, ystart, yend, clq)
    time2 = time.time()
    print(time2-time1)
    

In [73]:
horse = True
if horse:
    rundir = '/gpfs/data/greenocean/software/runs/'
    run = 'TOM12_TJ_1ASA'
    wrapper_ppt(rundir, run, 1998, 2010)
    
    run = 'TOM12_DW_WE43'
    wrapper_ppt(rundir, run, 1998, 2010)
    run = 'TOM12_DW_GA01'
    wrapper_ppt(rundir, run, 1998, 2010)
horse = True
if horse:
    run = 'LIG9'
    wrapper_ppt(rundir, run, 1998, 2010, clq = True)
    run = 'LIGD'    
    wrapper_ppt(rundir, run, 1998, 2010, clq = True)
    
    rundir = '/gpfs/home/hnb15vcu/scratch/ModelRuns/'
    run = 'TOM12_RW_3G20'
    wrapper_ppt(rundir, run, 1955, 1960)
    run = 'TOM12_RW_3G07'
    wrapper_ppt(rundir, run, 1955, 1960)
    run = 'TOM12_RW_3G09'
    wrapper_ppt(rundir, run, 1955, 1960)
    run = 'TOM12_RW_3G35'
    wrapper_ppt(rundir, run, 1955, 1960)

[0.12425972 0.09328803 0.05557739 0.03229137 0.01907872 0.01527927
 0.02262108 0.04301758 0.06233742 0.0816121  0.10074974 0.12156208]
[0.0042543  0.00341041 0.00264069 0.00218219 0.00166943 0.00128049
 0.00137659 0.00224213 0.00325448 0.00366041 0.0038888  0.00458316]
made u a netcdf bb:
./ncs/SOmap_ppt_TOM12_TJ_1ASA_1998_2010.nc
28.315321683883667
[0.08054769 0.06891702 0.04278507 0.02538021 0.01546662 0.01159792
 0.01591946 0.03029229 0.05067649 0.06513097 0.07051652 0.06927017]
[0.00348128 0.00258291 0.00187281 0.0014821  0.00113207 0.00085265
 0.00081459 0.00113798 0.00168616 0.00268137 0.0034289  0.00302437]
made u a netcdf bb:
./ncs/SOmap_ppt_TOM12_DW_WE43_1998_2010.nc
28.318439245224
[0.11760493 0.09086208 0.05883043 0.03265892 0.01676262 0.01170755
 0.01590473 0.03661636 0.07067304 0.09505919 0.10290185 0.11421795]
[0.00404229 0.00335253 0.00262142 0.00189395 0.00124847 0.00081403
 0.00076312 0.00145135 0.00309764 0.00439995 0.00433086 0.00446605]
made u a netcdf bb:
./ncs/SOm

In [54]:
w = xr.open_dataset('./ncs/SOmap_ppt_TOM12_RW_3G20_1955_1960.nc')
print(w)
print(w['ppt'])

<xarray.Dataset>
Dimensions:     (time: 12, x: 182, y: 37)
Coordinates:
    month       (time) int64 ...
  * y           (y) int64 0 1 2 3 4 5 6 7 8 9 ... 27 28 29 30 31 32 33 34 35 36
  * x           (x) int64 0 1 2 3 4 5 6 7 8 ... 174 175 176 177 178 179 180 181
Dimensions without coordinates: time
Data variables:
    ppt         (time) float64 ...
    EXP         (time) float64 ...
    ppt_summer  (y, x) float64 ...
    ppt_winter  (y, x) float64 ...
    EXP_summer  (y, x) float64 ...
    EXP_winter  (y, x) float64 ...
Attributes:
    description:  
<xarray.DataArray 'ppt' (time: 12)>
array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan])
Coordinates:
    month    (time) int64 ...
Dimensions without coordinates: time
Attributes:
    units:    (mol/m2/day top 200)


In [67]:
ylist = make_yearlist(1998,2000,'diad_T', run, rundir)
w = xr.open_mfdataset(ylist)
q = (w['EXP'][3,10,0:37,:])
q2 = -np.sort(np.ravel(-q))
print(q2[0:100])

[1.15656682e-07 1.15339638e-07 1.12905113e-07 1.11347667e-07
 1.10418789e-07 1.08712179e-07 1.08400911e-07 1.07946207e-07
 1.06746725e-07 1.06653928e-07 1.06552676e-07 1.06266306e-07
 1.05705645e-07 1.04786274e-07 1.04573964e-07 1.03879117e-07
 1.03814706e-07 1.02295701e-07 1.01486940e-07 1.01027339e-07
 1.00380170e-07 9.90628237e-08 9.83953683e-08 9.81664670e-08
 9.76074332e-08 9.66683587e-08 9.66385798e-08 9.47223597e-08
 9.45887280e-08 9.43065217e-08 9.42664826e-08 9.41706446e-08
 9.35196667e-08 9.29402191e-08 9.26865837e-08 9.25433881e-08
 9.15187357e-08 9.04896211e-08 9.04377018e-08 8.99694683e-08
 8.88967833e-08 8.88080649e-08 8.79823716e-08 8.65154703e-08
 8.56670397e-08 8.52125410e-08 8.51585327e-08 8.48771222e-08
 8.38891410e-08 8.37716314e-08 8.27153528e-08 8.16710610e-08
 8.14621615e-08 8.07176193e-08 7.89350523e-08 7.86346988e-08
 7.83343310e-08 7.79351481e-08 7.68709967e-08 7.66889983e-08
 7.63238575e-08 7.61923218e-08 7.56315615e-08 7.53910285e-08
 7.50911724e-08 7.495950